# Ejercicio Block Cipher
Diego Andrés Morales Aquino
21762


In [21]:
from Crypto.Cipher import DES
from Crypto.Random import get_random_bytes
import binascii

## 1. Generación una función cifrado y descifrado DES 

Código realizado con el apoyo del modelo GPT 3.5 <br>
Prompt disponible en: https://chatgpt.com/share/67ca43b1-7c64-8009-b4f7-8467992c94ff

In [ ]:

def pad(text):
    """
    Agregar padding. El padding está compuesto por el char que corresponde a la longitud del padding.
    """
    pad_length = 8 - len(text) % 8
    padding = chr(pad_length) * pad_length
    return text + padding

def des_encrypt(plain_text):

    key = get_random_bytes(8)
    padded_text = pad(plain_text)
    
    # Cifrado DES en modo ECB
    cipher = DES.new(key, DES.MODE_ECB)
    encrypted_text = cipher.encrypt(padded_text.encode())
    
    # Retornar llave y cifrado en hexa
    return binascii.hexlify(key).decode(), binascii.hexlify(encrypted_text).decode()

In [ ]:
def unpad(padded_text):
    """
    Obtener la longitud del padding a partir del ascii del último char y eliminarlo.
    """
    pad_length = ord(padded_text[-1])
    return padded_text[:-pad_length]

def des_decrypt(key_hex, encrypted_text_hex):

    # hex a bytes
    key = binascii.unhexlify(key_hex)
    encrypted_text = binascii.unhexlify(encrypted_text_hex)
    
    # Desencriptar con DES
    cipher = DES.new(key, DES.MODE_ECB)
    decrypted_text = cipher.decrypt(encrypted_text)
    
    # Eliminar el padding
    original_text = unpad(decrypted_text.decode())
    
    return original_text

In [24]:
# Encriptar texto
plain_text = "Mensaje de Diego Morales."
key, encrypted_text = des_encrypt(plain_text)

print(f"Llave generada: {key}")
print(f"Texto cifrado: {encrypted_text}")

Llave generada: 9991d3e6a2eb2774
Texto cifrado: b5645534d928298ca52ef2f8fb962b9fb6ff391212401389382b1a7eeaa31a09


In [25]:
# Desencriptar el texto
original_text = des_decrypt(key, encrypted_text)

print(f"Texto original: {original_text}")

Texto original: Mensaje de Diego Morales.
